# Chatbot using RAG with Memory

This notebook is to implement and create a RAG based chatbot that have memory. 

Reference: https://www.youtube.com/watch?v=PtO44wwqi0M

In [1]:
import os
from dotenv import load_dotenv

# Get api key
load_dotenv(".env")
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

## PyPDFLoader - pymupdf 

`pymupdf` Library that provides functionallity for loading PDF documents within the LangChain framework. `pymupdf`works better to interpret PDF's for LLM's.

In [2]:
from langchain_community.document_loaders import PyMuPDFLoader

In [3]:
# Loading the pdf document
loader = PyMuPDFLoader("C:/Users/Melissa Freitas/Documents/Melissa/BCGx-Challenge2024/data/Plano de Acao de Enfrentamento as Mudancas Climaticas do Grande ABC.pdf")


Now that we loaded the PDF, we can create variables that store each page as a separate chunk and stores the page nunmber in metadata

In [4]:
pages = loader.load_and_split()

In [5]:
pages[10].page_content

'9\nCapítulo 1: Introdução\nO Grande ABC localiza-se na sub-região sudeste \nda Região Metropolitana de São Paulo (RMSP), possui \nfronteiras regionais com os municípios de Mogi das \nCruzes, Suzano, Ferraz de Vasconcelos, São Paulo e as \ncidades da Baixada Santista. O Grande ABC é composto \npor sete municípios, Diadema, Mauá, Ribeirão Pires, Rio \nGrande da Serra, Santo André, São Bernardo do Campo \ne São Caetano do Sul (ver figura 1).\nO Grande ABC tem uma população de mais de \n2,7 milhões de habitantes, distribuídos em uma área \nterritorial de aproximadamente 828 km² 3. As tabelas a \nseguir apresentam as características demográficas de \ncada município, suas respectivas extensões territoriais e \no Índice de Desenvolvimento Humano Municipal (IDHM), \nevidenciado a composição heterogênea do Grande ABC. \nAo se analisar a região sob a ótica sub-regional, obtêm-\nse resultados diferentes daqueles de nível local, não \nevidenciando de forma adequada as particularidades \nsocioecon

Since each page of the PDF is still quite long, we would like to break the pages into smaller pieces giving a bit of overlap so that no meaningful sentence get lost.

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(pages)

In [7]:
print(f"{len(pages)} vs {len(documents)}")

68 vs 211


## Embbedings Creation

We are going to use OpenAI embbedings to convert each chunk of text to numeric vectors.

In [8]:
from langchain_openai import OpenAIEmbeddings
embbedings = OpenAIEmbeddings(openai_api_type=OPENAI_API_KEY)

### Chroma Vector Database

We are going to store all embbefings (numeric vectors) on one local database that is Chroma Vector DB (similar to FAISS DB)

In [9]:
from langchain_community.vectorstores import FAISS

faiss = FAISS.from_documents(documents, embbedings)

In [10]:
from langchain_core.vectorstores import VectorStoreRetriever

retriever = faiss.as_retriever(search_type="similarity_score_threshold", 
                               search_kwargs={"score_threshold": 0.5},
                               search_limit=5
)

## Large Language Model

In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, 
                 model="gpt-3.5-turbo",
                 temperature=0.5, 
                 max_tokens=1000)

## Adding Memory

One user asks a new question, there is a history of questions and answers in their mind. Here, the idea is to reformulate user's question into a format that has its own context.

We are going to use LLM to perform this reformulation of the question in a way that this reformuled question uses the conversation history.

When building a RAG-based chatbot for municipal workers such as the mayor, councilors, and technical specialists, it is crucial to maintain a coherent and contextually aware conversation. The best memory option for this scenario is a combination of `ConversationSummaryMemory` and `ConversationEntityMemory`.

1- **ConversationSummaryMemory**:
    - **Purpose**: This memory type helps in summarizing the conversation history, making it easier to keep track of the main points discussed without overloading the system with too much detail.
    - **Benefit**: It ensures that the chatbot can provide concise and relevant responses based on the summarized context, which is particularly useful for busy municipal workers who need quick and accurate information.

2 - **ConversationEntityMemory**:
    - **Purpose**: This memory type focuses on tracking specific entities mentioned during the conversation, such as names, dates, locations, and other important details.
    - **Benefit**: It allows the chatbot to remember and reference key entities accurately, ensuring that the conversation remains contextually rich and relevant. This is essential for technical specialists who may need to discuss specific projects, plans, or data points.

By combining these two memory types, the chatbot can maintain a high-level summary of the conversation while also keeping track of important entities. This dual approach ensures that the chatbot remains both contextually aware and detail-oriented, providing municipal workers with the information they need to make informed decisions about climate change and sustainability plans.

In [12]:
from langchain.memory import CombinedMemory, ConversationSummaryMemory, ConversationEntityMemory
from langchain_openai import OpenAI

# Create the summarization memory, using a custom memory key
summary_memory = ConversationSummaryMemory(llm=OpenAI(), memory_key="summary_history")  # Use 'summary_history' as key

# Create the entity memory, using a different memory key
entity_memory = ConversationEntityMemory(llm=OpenAI(), memory_key="entity_info")  # Use 'entity_info' as key

# Combine both
# memories
memory = CombinedMemory(memories=[summary_memory, entity_memory])

C:\Users\Melissa Freitas\AppData\Local\Temp\ipykernel_41448\4270948597.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  summary_memory = ConversationSummaryMemory(llm=OpenAI(), memory_key="summary_history")  # Use 'summary_history' as key
C:\Users\Melissa Freitas\AppData\Local\Temp\ipykernel_41448\4270948597.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  entity_memory = ConversationEntityMemory(llm=OpenAI(), memory_key="entity_info")  # Use 'entity_info' as key
c:\Users\Melissa Freitas\Documents\Melissa\BCGx-Challenge2024\.venv\Lib\site-packages\pydantic\main.py:212: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
c:\Users\Melissa Freitas\Documents\Melissa

### Prompt Definition and Chat

In [13]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import ConversationChain
from langchain import LLMChain 

In [14]:
# General conversation prompt
instructions = """
Given the chat history and the latest user question, which might reference context in the chat history,
formulate a standalone question, which can be undestood whithout the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is
"""

In [15]:
question_maker_prompt = ChatPromptTemplate(
    [
        ("system", instructions),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

question_chain = LLMChain(
    llm=llm,
    prompt=question_maker_prompt,
    memory=memory
)


C:\Users\Melissa Freitas\AppData\Local\Temp\ipykernel_41448\2058542466.py:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  question_chain = LLMChain(


#### PROMPT

We now build the prompt for the question and answer.

This time, the prompt consists of:
- A python-list os fystem instruction
- A place holder to take the caht history later on
- user's question

In [16]:
qa_system_prompt = """
    You are an assistant, a climate crisis specialist, and you're answering questions about the document. 
    Use the following retrieved information, as context, to answer the user's question.
    
    Context: {context}
    
    Question: {question}
"""

# Define the prompt structure
qa_prompt = ChatPromptTemplate.from_template(qa_system_prompt)

In [17]:
def contextualized_question(input: dict):
    if input.get("chat_history"):
        return question_chain.invoke(input)
    else:
        return {"question": input['question']}

### Retriever Chain

We need a chain to pass the following to the LLM:
- **Context**: Use the vector retriever and get the most relevant chunks of the PDF.
- **Question**: Reformulated or the original user's question depending on the history
- **Chat_History**: Python list of the chats

We use the assign function which adds the context to whatever it gets as input and pass it to the next link of the chain

In [18]:
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import ConversationalRetrievalChain

In [19]:
format_docs = RunnablePassthrough()

In [20]:
# Build the conversational retrieval chain
retriever_chain = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    retriever=retriever, 
    return_source_documents=True  
)

## Retrieve-Augmented Generation (RAG) Chain: The man chain

This is the main chain that produces the final answer

In [21]:
from langchain.schema import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser


In [22]:
rag_chain = (
        retriever_chain
        | qa_prompt
        | llm
        | StrOutputParser()
    )

In [29]:
question = "Qual é o titulo do documento?"
chat_history = [HumanMessage(content="Sabendo que o título é, geralmente, o text na primeira página.")]
context = retriever.get_relevant_documents(question)


if context:  # Only invoke if context is not empty
    ai_message = rag_chain.invoke({
        "question": question,
        "context": context,
        "chat_history": chat_history
    })

    chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_message)])

    print(ai_message)
    for doc in context:
        print(f"Number of Document Page: \n{doc.metadata.get('page', 'Unknown')}")
        print(f"Page Content used as Reference: \n{doc.page_content}")
        print("\n\n ------------------- \n\n")
else:
    print("No relevant context available for the question.")
    

O título do documento não foi especificado na informação fornecida. Seria necessário revisar o conteúdo do documento para identificar o título.
Number of Document Page: 
36
Page Content used as Reference: 
represa Billings, de acordo com o Decreto Estadual nº. 
10.755/1977.


 ------------------- 


Number of Document Page: 
27
Page Content used as Reference: 
dades, interesses e capacidades técnicas e financeira 
do contexto regional. Espera-se que, dessa forma, o 
Consórcio contribua para que tanto o estado de São 
Paulo, quanto o Governo Nacional possam cumprir 
seus compromissos legais para enfrentar as mudanças 
climáticas.
Energia
15O documento de análise para as emissões do setor de energia, processos industriais e uso de produtos está 
disponível em: goo.gl/0gsbeN
16Fonte Novos Padrões de Qualidade do Ar: Disponível em: goo.gl/fvah8U


 ------------------- 


Number of Document Page: 
7
Page Content used as Reference: 
ABC.
Luiz Marinho
Presidente do Consórcio Intermunicipal Gr

In [30]:
question = "O que grande ABC significa?"
context = retriever.get_relevant_documents(question)
ai_message = rag_chain.invoke({"question": question, "context": context, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_message)])

print(ai_message)
for doc in context:
    print(f"Number of Document Page: \n{doc.metadata.get('page', 'Unknown')}")
    print(f"Page Content used as Reference: \n{doc.page_content}")
    print("\n\n ------------------- \n\n")

Grande ABC significa Grande Aglomeração Urbana do ABC, que se refere à região formada por sete municípios: Diadema, Mauá, Ribeirão Pires, Rio Grande da Serra, Santo André, São Bernardo do Campo e São Caetano do Sul.
Number of Document Page: 
9
Page Content used as Reference: 
9
Capítulo 1: Introdução
O Grande ABC localiza-se na sub-região sudeste 
da Região Metropolitana de São Paulo (RMSP), possui 
fronteiras regionais com os municípios de Mogi das 
Cruzes, Suzano, Ferraz de Vasconcelos, São Paulo e as 
cidades da Baixada Santista. O Grande ABC é composto 
por sete municípios, Diadema, Mauá, Ribeirão Pires, Rio 
Grande da Serra, Santo André, São Bernardo do Campo 
e São Caetano do Sul (ver figura 1).
O Grande ABC tem uma população de mais de 
2,7 milhões de habitantes, distribuídos em uma área 
territorial de aproximadamente 828 km² 3. As tabelas a 
seguir apresentam as características demográficas de 
cada município, suas respectivas extensões territoriais e 
o Índice de Desenvolvime

In [31]:
question = "A resposta que você forneceu sobre as cidades do Grande ABC foi baseada em dados obtidos no documento ou você respondeu com base em seu conhecimento prévio?"
context = retriever.get_relevant_documents(question)
ai_message = rag_chain.invoke({"question": question, "context": context, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_message)])

print(ai_message)
for doc in context:
    print(f"Number of Document Page: \n{doc.metadata.get('page', 'Unknown')}")
    print(f"Page Content used as Reference: \n{doc.page_content}")
    print("\n\n ------------------- \n\n")

A resposta que forneci sobre as cidades do Grande ABC foi baseada em dados obtidos no documento "Plano de Ação de Enfrentamento às Mudanças Climáticas do Grande ABC". Este documento fornece informações detalhadas sobre a região, sua composição, características demográficas, e impactos das atividades industriais na região. Portanto, minha resposta foi fundamentada nos dados contidos no documento, e não em conhecimento prévio.
Number of Document Page: 
7
Page Content used as Reference: 
ABC.
Luiz Marinho
Presidente do Consórcio Intermunicipal Grande ABC 
Prefeito de São Bernardo do Campo
Apresentação¹
¹O Plano de Ação é um documento baseado no Primeiro Inventário Regional de Emissões de Gases 
de Efeito Estufa do Grande ABC. Em ambos,  a Apresentação e Contexto são os mesmos.


 ------------------- 


Number of Document Page: 
9
Page Content used as Reference: 
9
Capítulo 1: Introdução
O Grande ABC localiza-se na sub-região sudeste 
da Região Metropolitana de São Paulo (RMSP), possui 
fr

#### Which question pass to LLM?

We define a function that looks at the chat history,
- If there is a history: It will pass the question chain (that reformulates the user's question)
- If chat history is empty, it will pass the user's question directly

In [32]:
question = "Forneça em tópicos um resumo de todas as estratégias abordadas no documento"
context = retriever.get_relevant_documents(question)
ai_message = rag_chain.invoke({"question": question, "context": context, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_message)])

print(ai_message)
for doc in context:
    print(f"Number of Document Page: \n{doc.metadata.get('page', 'Unknown')}")
    print(f"Page Content used as Reference: \n{doc.page_content}")
    print("\n\n ------------------- \n\n")

- Estabelecimento de políticas consistentes e articulação de uma rede de parceiros estratégicos para ampliar o acesso a recursos.
- Cooperação entre diferentes níveis de governo e parcerias com outras cidades pelo mundo.
- Revisão e aprimoramento da execução da estratégia para garantir a continuidade no caminho correto.
- Divulgação da agenda e inspiração para os próximos passos.
- Identificação de quatro setores prioritários para o combate às mudanças climáticas no Grande ABC: Energia, Transporte e mobilidade urbana, Resíduos e saneamento, Desenvolvimento urbano sustentável e resiliente.
- Sugestão para os stakeholders locais acompanharem iniciativas em outras cidades do Brasil e do mundo, visando replicar iniciativas positivas e transição para a economia de baixo carbono.
- Definição de metas e submetas para cada setor estratégico, como medidas de compostagem e reciclagem.
Number of Document Page: 
18
Page Content used as Reference: 
municipal, por meio da colaboração com outras cida

In [34]:
chat_history

[HumanMessage(content='Sabendo que o título é, geralmente, o text na primeira página.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Qual é o titulo do documento?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='O título do documento não foi especificado na informação fornecida. Seria necessário revisar o conteúdo do documento para identificar o título.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='O que grande ABC significa?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Grande ABC significa Grande Aglomeração Urbana do ABC, que se refere à região formada por sete municípios: Diadema, Mauá, Ribeirão Pires, Rio Grande da Serra, Santo André, São Bernardo do Campo e São Caetano do Sul.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='A resposta que você forneceu sobre as cidades do Grande ABC foi baseada em dados obtidos no documento ou você respondeu com base em seu conhecimento pré

In [35]:
question = "Quais são os parceiros estratégicos ctados no primeiro tópico?"
context = retriever.get_relevant_documents(question)
ai_message = rag_chain.invoke({"question": question, "context": context, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_message)])

print(ai_message)
for doc in context:
    print(f"Number of Document Page: \n{doc.metadata.get('page', 'Unknown')}")
    print(f"Page Content used as Reference: \n{doc.page_content}")
    print("\n\n ------------------- \n\n")

No primeiro tópico do documento, os parceiros estratégicos citados são os municípios que aderiram ao Compacto de Prefeitos, as pesquisas e avaliações do contexto intermunicipal, e os stakeholders locais que participaram das discussões e oficinas sobre o plano de ação de enfrentamento às mudanças climáticas do Grande ABC.
Number of Document Page: 
27
Page Content used as Reference: 
dades, interesses e capacidades técnicas e financeira 
do contexto regional. Espera-se que, dessa forma, o 
Consórcio contribua para que tanto o estado de São 
Paulo, quanto o Governo Nacional possam cumprir 
seus compromissos legais para enfrentar as mudanças 
climáticas.
Energia
15O documento de análise para as emissões do setor de energia, processos industriais e uso de produtos está 
disponível em: goo.gl/0gsbeN
16Fonte Novos Padrões de Qualidade do Ar: Disponível em: goo.gl/fvah8U


 ------------------- 


Number of Document Page: 
55
Page Content used as Reference: 
fornecer elementos e subsídios relev

In [36]:
question = "Como as estratégias citadas no documento sera implementadas?"
context = retriever.get_relevant_documents(question)
ai_message = rag_chain.invoke({"question": question, "context": context, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_message)])

print(ai_message)
for doc in context:
    print(f"Number of Document Page: \n{doc.metadata.get('page', 'Unknown')}")
    print(f"Page Content used as Reference: \n{doc.page_content}")
    print("\n\n ------------------- \n\n")

As estratégias citadas no documento serão implementadas através de um processo detalhado em várias etapas, conforme descrito no plano de ação de enfrentamento às mudanças climáticas do Grande ABC. 

1. Passo 5: Detalhamento e financiamento - Neste passo, as estratégias serão detalhadas com mais profundidade, avaliando sua eficácia e viabilidade técnica e financeira. Isso permitirá testar e demonstrar a capacidade adaptativa e de redução de emissões de GEE das novas ações. Projetos com financiamento assegurado poderão ser postos em prática.

2. Passo 6: Implementação e monitoramento - A execução será feita em parceria com organizações selecionadas, e cada projeto será monitorado e avaliado, com mensuração, reporte e verificação, para atender às condições dos financiadores ou para documentação de maneira voluntária.

3. Passo 7: Integração e colaboração - Neste passo, será integrada a política e infraestrutura urbanas para além da jurisdição municipal, colaborando com outras cidades para

In [37]:
question = "VocÊ começou a descrever a partir do Passo 5. Quais são os passos anteriores?"
context = retriever.get_relevant_documents(question)
ai_message = rag_chain.invoke({"question": question, "context": context, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_message)])

print(ai_message)
for doc in context:
    print(f"Number of Document Page: \n{doc.metadata.get('page', 'Unknown')}")
    print(f"Page Content used as Reference: \n{doc.page_content}")
    print("\n\n ------------------- \n\n")

Os passos anteriores ao Passo 5, que foi descrito no documento, são os seguintes:

1. Passo 1: Avaliação interna das fraquezas e fortalezas do município, considerando questões sociais, econômicas, ambientais e políticas, além de uma avaliação externa das oportunidades e obstáculos.

2. Passo 2: Linha de base, que inclui o inventário de emissões de GEE e a análise de vulnerabilidade.

3. Passo 3: Desenvolvimento da estratégia municipal, por meio da colaboração com outras cidades, para melhorar a qualidade e execução dos programas.

4. Passo 4: Revisão e aprimoramento, avaliando a execução da estratégia para garantir que a cidade continue no caminho certo.

5. Passo 5: Divulgação da agenda e inspiração, finalizando o processo de formulação da estratégia e preparando para os próximos passos.
Number of Document Page: 
18
Page Content used as Reference: 
18
Plano de Ação de Enfrentamento às Mudanças Climáticas do Grande ABC
Passo 5: 
Detalhamento e 
financiamento
O momento agora é o de deta

In [38]:
question = "Porque na primeira vez você iniciou a resposta a partir do passo 5 e quando questionado falou dos passos anteriores? Porque você não começou a resposta a partir do passo 1?"
context = retriever.get_relevant_documents(question)
ai_message = rag_chain.invoke({"question": question, "context": context, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_message)])

print(ai_message)
for doc in context:
    print(f"Number of Document Page: \n{doc.metadata.get('page', 'Unknown')}")
    print(f"Page Content used as Reference: \n{doc.page_content}")
    print("\n\n ------------------- \n\n")

Na primeira vez, iniciei a resposta a partir do Passo 5 porque o contexto fornecido mencionava a etapa de detalhamento e financiamento do Plano de Ação de Enfrentamento às Mudanças Climáticas do Grande ABC. Ao ser questionado sobre os passos anteriores, como o Passo 1, eu incluí informações sobre essas etapas para fornecer uma visão mais abrangente do processo de formulação e implementação do plano. Comecei a resposta a partir do Passo 5 porque foi o ponto de referência inicial fornecido, mas posso fornecer informações sobre os passos anteriores, como o Passo 1, se necessário.
Number of Document Page: 
54
Page Content used as Reference: 
Betim 
Não há
-   
Não elaborou um plano de ação.


 ------------------- 


Number of Document Page: 
18
Page Content used as Reference: 
18
Plano de Ação de Enfrentamento às Mudanças Climáticas do Grande ABC
Passo 5: 
Detalhamento e 
financiamento
O momento agora é o de detalhar a estratégia com mais profundidade, 
avaliar sua eficácia e viabilidades 

In [39]:
question = "Hello chat, how are you doing?"
context = retriever.get_relevant_documents(question)
ai_message = rag_chain.invoke({"question": question, "context": context, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_message)])

print(ai_message)
for doc in context:
    print(f"Number of Document Page: \n{doc.metadata.get('page', 'Unknown')}")
    print(f"Page Content used as Reference: \n{doc.page_content}")
    print("\n\n ------------------- \n\n")

Hello! I'm here and ready to assist you. How can I help you today?
Number of Document Page: 
3
Page Content used as Reference: 
Marina Gomes – Estagiária
Sophia Bujnicki Picarelli – Gerente de Projetos	
PROJETO GRÁFICO E DIAGRAMAÇÃO
Nathany Paola da Silva
REVISÃO DE TEXTO
P&B Comunicação


 ------------------- 


Number of Document Page: 
1
Page Content used as Reference: 
PLANO DE AÇÃO DE 
ENFRENTAMENTO 
ÀS MUDANÇAS CLIMÁTICAS 
DO GRANDE ABC


 ------------------- 


Number of Document Page: 
50
Page Content used as Reference: 
Grande ABC
Brasil 
9.879.277
2.702.071
4
27
60
12
14
Ribeirão Pires
Brasil 
426.875
119.644
3,57
12
75
13
15
Paris
França
6.190.189
2.265.886
2,73
76
17
8
16
Mauá
Brasil 
1.197.707
448.776
2,67
33
54
13
17
Diadema
Brasil 
1.087.960
409.613
2,66
27
57
15
18
Rio de Janeiro 
Brasil 
15.856.906
6.453.682
2,46
51
34
15
19
Quito
Equador
5.168.800
2.239.191
2,31
48
32
20
20
Oslo
Noruega
1.340.422
647.676
2,07
38
58
4
21
Recife
Brasil 
3.120.426
1.555.039
2,01
15
66
19

In [40]:
question = "Meu nome é Melissa e eu gostaria que você pensasse em um apelido para mim e o usasse toda vez que fosse responder uma pergunta, como se fôssemos amigos"
context = retriever.get_relevant_documents(question)
ai_message = rag_chain.invoke({"question": question, "context": context, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_message)])

print(ai_message)
for doc in context:
    print(f"Number of Document Page: \n{doc.metadata.get('page', 'Unknown')}")
    print(f"Page Content used as Reference: \n{doc.page_content}")
    print("\n\n ------------------- \n\n")

Claro, Melissa! Vou te chamar de Mel, como se já fossemos amigos. Se precisar de mais alguma coisa, é só me chamar, Mel!
Number of Document Page: 
3
Page Content used as Reference: 
Marina Gomes – Estagiária
Sophia Bujnicki Picarelli – Gerente de Projetos	
PROJETO GRÁFICO E DIAGRAMAÇÃO
Nathany Paola da Silva
REVISÃO DE TEXTO
P&B Comunicação


 ------------------- 


Number of Document Page: 
2
Page Content used as Reference: 
João Ricardo Guimarães Caetano
Coordenador
 
Grupo de Trabalho Defesa Civil
Debora Diogo 
Coordenadora
Grupo Temático Mudanças Climáticas
Vanessa Valente – São Bernardo do Campo
Coordenadora
Debora Diogo – Santo André
Coordenadora   
Débora Duarte Stefanelli – Santo André
Maira Galvanese – Santo André
Gabriela Priolli de Oliveira – São Bernardo do Campo
Horácio Pires – São Caetano do Sul
Francisco Emilio Peres Schulze – Diadema


 ------------------- 


Number of Document Page: 
2
Page Content used as Reference: 
EXPEDIENTE
CONSÓRCIO INTERMUNICIPAL GRANDE ABC
ASSE

In [41]:
question = "Quando eu pedi que você me chamasse pelo meu apelido você não precisava acessar os documentos para responder e ainda assim alguns documentos foram devolvidos do RAG. Pq isso aocnteceu?"
context = retriever.get_relevant_documents(question)
ai_message = rag_chain.invoke({"question": question, "context": context, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_message)])

print(ai_message)
for doc in context:
    print(f"Number of Document Page: \n{doc.metadata.get('page', 'Unknown')}")
    print(f"Page Content used as Reference: \n{doc.page_content}")
    print("\n\n ------------------- \n\n")

Desculpe pela confusão, mas parece que houve um erro na interpretação da sua pergunta. Eu não acessei nenhum documento para responder à sua solicitação de chamá-lo pelo apelido. A devolução de documentos do RAG pode ter ocorrido devido a um erro de sistema ou configuração. Se precisar de mais alguma informação, estou à disposição para ajudar.
Number of Document Page: 
56
Page Content used as Reference: 
Para o caso de considerá-las, será preciso revisar tanto 
o inventário de 2014 como o Cenário de Referência.


 ------------------- 


Number of Document Page: 
54
Page Content used as Reference: 
Betim 
Não há
-   
Não elaborou um plano de ação.


 ------------------- 


Number of Document Page: 
37
Page Content used as Reference: 
arcabouço legal bem instrumentalizado e diagnósticos 
técnicos completos sobre a existência de processos, há 
uma deficiência no enfoque para a redução de gases de 
efeito estufa, tanto no tratamento aplicado aos resíduos 
sólidos, como também no saneamento 

: 